# notebook_ragblog

This notebook allows to run ragblog/run.py from google colab. The objective is to access GPU backend for the llm workload.



In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!curl -sSL https://install.python-poetry.org | python3 -


Retrieving Poetry metadata

The latest version (1.8.3) is already installed.


In [3]:
# !export PATH="/root/.local/bin:$PATH"

In [4]:
import os
os.environ["PATH"] += os.pathsep + os.path.expanduser("~/.local/bin")


In [5]:
! rm -r ml-sandbox
! git clone https://github.com/adrianmatias/ml-sandbox.git

Cloning into 'ml-sandbox'...
remote: Enumerating objects: 201, done.
remote: Counting objects: 100% (201/201), done.
remote: Compressing objects: 100% (153/153), done.
remote: Total 201 (delta 68), reused 165 (delta 40), pack-reused 0
Receiving objects: 100% (201/201), 3.20 MiB | 5.43 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [6]:
os.chdir("ml-sandbox/ragblog")

In [7]:
!ls

lint.sh  poetry.lock  pyproject.toml  ragblog  README.md  tests


In [8]:
!poetry install

Installing dependencies from lock file

No dependencies to install or update

Installing the current project: ragblog (0.1.0)Installing the current project: ragblog (0.1.0)


In [9]:
!poetry show --tree


beautifulsoup4 4.12.3 Screen-scraping library
└── soupsieve >1.2
black 24.4.2 The uncompromising code formatter.
├── click >=8.0.0
│   └── colorama * 
├── mypy-extensions >=0.4.3
├── packaging >=22.0
├── pathspec >=0.9.0
├── platformdirs >=2
├── tomli >=1.1.0
└── typing-extensions >=4.0.1
isort 5.13.2 A Python utility / library to sort Python imports.
jq 1.7.0 jq is a lightweight and flexible JSON processor.
langchain 0.2.11 Building applications with LLMs through composability
├── aiohttp >=3.8.3,<4.0.0
│   ├── aiosignal >=1.1.2 
│   │   └── frozenlist >=1.1.0 
│   ├── async-timeout >=4.0,<5.0 
│   ├── attrs >=17.3.0 
│   ├── frozenlist >=1.1.1 (circular dependency aborted here)
│   ├── multidict >=4.5,<7.0 
│   └── yarl >=1.0,<2.0 
│       ├── idna >=2.0 
│       └── multidict >=4.0 (circular dependency aborted here)
├── async-timeout >=4.0.0,<5.0.0
├── langchain-core >=0.2.23,<0.3.0
│   ├── jsonpatch >=1.33,<2.0 
│   │   └── jsonpointer >=1.9 
│   ├── langsmith >=0.1.75,<0.2.0 
│   

In [10]:
# allow ollama to access colab bakend's GPU
! sudo apt install pciutils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pciutils is already the newest version (1:3.7.0-6).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [11]:
! lspci --version

lspci version 3.7.0


In [12]:
! curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [13]:
! nohup ollama serve > myoutput.txt >2&1


/bin/bash: line 1: 1: command not found
nohup: redirecting stderr to stdout


In [14]:
! ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 970aa74c0a90... 100% ▕▏ 274 MB                         
pulling c71d239df917... 100% ▕▏  11 KB                         
pulling ce4a164fc046... 100% ▕▏   17 B                         
pulling 31df23ea7daa... 100% ▕▏  420 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [15]:
! ollama pull llama3.1:8b
# ! ollama run llama3.1:70b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling 87048bcd5521... 100% ▕▏ 4.7 GB                         
pulling 8cf247399e57... 100% ▕▏ 1.7 KB                         
pulling f1cd752815fc... 100% ▕▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕▏   96 B                         
pulling e711233e7343... 100% ▕▏  485 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [16]:
import sys
poetry_env_path = !poetry env info --path
poetry_env_path = poetry_env_path[0]  # Get the first line of the output
sys.path.append(os.path.join(poetry_env_path, 'lib/python3.10/site-packages'))
poetry_env_path

'/root/.cache/pypoetry/virtualenvs/ragblog-SSGc0OJ3-py3.10'

In [17]:
# ! rm -r data
# ! rm -r chroma

In [18]:
! mkdir -p data
! mkdir -p chroma

In [19]:
import os

from ragblog.conf import CONF
from ragblog.crawler import ConfCrawler, Crawler
from ragblog.logger_custom import LoggerCustom
from ragblog.rag_pipeline import (
    JSONLoaderConf,
    RAGChainConf,
    RagPipeline,
    RagPipelineConf,
    TextSplitterConf,
    VectorStoreConf,
)


def main():

    crawler = Crawler(conf_crawler=ConfCrawler(post_count_min=1000))
    crawler.get_url_list()
    crawler.get_post_list()
    crawler.write(path=CONF.path.data)

    rp_conf = RagPipelineConf(
        loader=JSONLoaderConf(
            file_path=os.path.join(CONF.path.data, "blog.jsonl"),
            jq_schema=".text",
            text_content=False,
            json_lines=True,
        ),
        splitter=TextSplitterConf(chunk_size=1000, chunk_overlap=10),
        vectorstore=VectorStoreConf(
            embedding_model="nomic-embed-text",
            persist_directory=CONF.path.chroma,
        ),
        ragchain=RAGChainConf(
            prompt_model="rlm/rag-prompt-llama", llm_model="llama3.1:8b"
        ),
        is_db_ready=False,
        is_debug=False,
    )

    pipeline = RagPipeline(conf=rp_conf, logger=LoggerCustom().get_logger())
    response = pipeline.query(
        question="""Describe the relation between Helena and Alejandra. Consider the author's diverse experiences and
        multifaceted personality, reflecting on traits that are evident across their various blog posts. Provide a
        detailed and thoughtful response. Ensure your answer
        is profound and sufficiently long, offering deep insights and personal reflections."""
    )
    print(response)


if __name__ == "__main__":
    main()


2024-08-01 10:48:08,272,272 INFO     [crawler.py:__init__:27]. self.__dict__: {'conf': ConfCrawler(url='https://delightfulobservaciones.blogspot.com/', post_count_min=1000), 'url_list': [], 'post_list': []}
2024-08-01 10:48:08,274,274 INFO     [crawler.py:get_url_list:37]. Fetching https://delightfulobservaciones.blogspot.com/
2024-08-01 10:48:08,863,863 INFO     [crawler.py:get_url_list:49]. url: https://delightfulobservaciones.blogspot.com/2024/03/sol-de-mi-corazon.html
2024-08-01 10:48:08,865,865 INFO     [crawler.py:get_url_list:49]. url: https://delightfulobservaciones.blogspot.com/2024/03/el-tao-de-ser-padre.html
2024-08-01 10:48:08,868,868 INFO     [crawler.py:get_url_list:49]. url: https://delightfulobservaciones.blogspot.com/2023/10/poco-antes-de-alejandra.html
2024-08-01 10:48:08,870,870 INFO     [crawler.py:get_url_list:49]. url: https://delightfulobservaciones.blogspot.com/2020/04/la-materializacion-del-riesgo.html
2024-08-01 10:48:08,872,872 INFO     [crawler.py:get_url_li